In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
import os

from tensorflow.keras.optimizers import Adam
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
# 加载 CIFAR-10 数据
import random
import numpy as np
# Load and preprocess CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


2024-07-22 16:07:39.260927: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [2]:
##该模型并没有使用双重对齐，因为我使用的数据错了。

In [3]:


class sigTriggerAttack(object):
    def __init__(self, delta=40, f=6):
        self.delta = delta
        self.f = f

    def __call__(self, img):
        return self.sigTrigger(img)

    def sigTrigger(self, img):
        img = np.float32(img)
        pattern = np.zeros_like(img)
        m = pattern.shape[1]
        for i in range(int(img.shape[0])):
            for j in range(int(img.shape[1])):
                pattern[i, j] = self.delta * np.sin(2 * np.pi * j * self.f / m)

        img = np.uint32(img * 255) + pattern  # 因为您之前已经将图像归一化，所以我们需要乘以255
        img = np.uint8(np.clip(img, 0, 255))
        return img / 255  # 再次归一化

    
    
# Apply the SIG backdoor attack
# Apply the SIG backdoor attack to the training set
attack = sigTriggerAttack()

# 选择1%的训练数据进行攻击
num_samples = int(1/32* x_val.shape[0])
indices = np.random.choice(y_val.shape[0], num_samples, replace=False)

target_class = 1  # 目标类别
num_classes = 10  # CIFAR-10 数据集的类别数

# 新的数据集和标签列表
x_val_triggered = []  # 存储触发器转换后的图像
y_val_triggered = []  # 存储更新后的标签

x_val_tigger = x_val.copy()
y_val_tigger = y_val.copy()

# 应用攻击到选中的图像
for i in indices:
    x_val_tigger[i] = attack(x_val_tigger[i])
    y_val_tigger[i] = to_categorical(target_class, num_classes=num_classes)
    x_val_triggered.append(x_val_tigger[i])
    y_val_triggered.append(to_categorical(target_class, num_classes=num_classes))
    
# 将列表转换为NumPy数组（如果需要）
x_val_triggered = np.array(x_val_triggered)
y_val_triggered = np.array(y_val_triggered)
        
    
    
x_test_tigger = x_test.copy()
y_test_tigger = y_test.copy()

for i in range(len(x_test_tigger)):
    x_test_tigger[i] = attack(x_test_tigger[i])
    y_test_tigger[i] = to_categorical(target_class, num_classes=num_classes)


In [4]:
##进行攻击的思路
#1.将数据集分成两部分  8比2吧
#2.每个epcoch后再单独优化一下服务器模型，保持模拟的客户端模型不变，后门数据要多一些
#3.验证数据集结果

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models



def create_client_model(input_shape):
    input_layer = layers.Input(shape=input_shape)
    
    # 第一层卷积
    x = layers.Conv2D(32, 3, strides=1, padding='same')(input_layer)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # 第二层卷积
    x = layers.Conv2D(64, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # 新增的第三层卷积
    x = layers.Conv2D(128, 3, strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    model = models.Model(inputs=input_layer, outputs=x)
    return model


In [6]:
from tensorflow.keras import layers, models

def res_block(filters, strides):
    def block(x):
        shortcut = x

        x = layers.Conv2D(filters, 3, padding='same', strides=strides)(x)
        x = layers.BatchNormalization()(x)
        x = layers.Activation('relu')(x)

        x = layers.Conv2D(filters, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        # 捷径连接前的调整
        shortcut = layers.Conv2D(filters, 1, strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

        x = layers.add([x, shortcut])
        x = layers.Activation('relu')(x)
        return x
    return block

def create_server_model():
    # 调整输入层的定义以匹配修改后的客户端模型的输出
    input_layer = layers.Input(shape=(32, 32, 128))  # 注意这里的变化

    # 残差块定义保持不变，继续使用提前定义的 res_block
    x = res_block(128, 1)(input_layer)  # 使用第一个残差块
    x = res_block(128, 2)(x)            # 使用第二个残差块
    x = res_block(128, 2)(x)           # 使用第三个残差块
    x = res_block(256, 2)(x)           # 使用第四个残差块

    # 全局平均池化和输出层保持不变
    x = layers.GlobalAveragePooling2D()(x)
    output = layers.Dense(10, activation='softmax')(x)
    
    model = models.Model(inputs=input_layer, outputs=output)
    return model





In [7]:
# 创建客户端模型
client_model = create_client_model(input_shape=(32, 32, 3))

# 编译客户端模型
client_model.compile(optimizer=Adam(),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])


2024-07-22 16:09:11.868582: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2024-07-22 16:09:11.967691: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: NVIDIA Tesla V100-PCIE-16GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2024-07-22 16:09:11.967758: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2024-07-22 16:09:11.982860: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2024-07-22 16:09:11.988881: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2024-07-22 16:09:11.989449: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcu

In [8]:
client_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)       

In [9]:
# 以下是服务器模型训练的简化示例
server_model = create_server_model()
server_model.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

In [10]:
##创建三个影子模型，分别是1层卷积，2层卷积核3层卷积
server_model_1 = create_server_model()
server_model_1.compile(optimizer=Adam(),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

In [11]:
# Assuming TensorFlow is installed and the functions are defined in your script
input_shape = (32, 32, 3)

client_model_1 = create_client_model(input_shape=(32, 32, 3))

# You can then print the model summaries to verify their structures
print(client_model_1.summary())




Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_27 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
activation_19 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
batch_normalization_28 (Batc (None, 32, 32, 64)        256       
_________________________________________________________________
activation_20 (Activation)   (None, 32, 32, 64)       

In [12]:
@tf.function
def train_step_client(x, y):
    # 使用客户端和服务器模型进行训练
    with tf.GradientTape() as tape_client, tf.GradientTape() as tape_server:
        # 客户端模型前向传播
        client_outputs = client_model(x, training=True)
        # 服务器模型前向传播
        server_logits = server_model(client_outputs, training=True)
        # 计算损失
        loss = loss_fn(y, server_logits)
    
    # 计算并应用服务器模型梯度
    grads_server = tape_server.gradient(loss, server_model.trainable_variables)
    optimizer_server.apply_gradients(zip(grads_server, server_model.trainable_variables))
    
    # 计算并应用客户端模型梯度
    grads_client = tape_client.gradient(loss, client_model.trainable_variables)
    optimizer_client.apply_gradients(zip(grads_client, client_model.trainable_variables))

    return loss


In [13]:
def train_step_client_1(x, y):
    # 使用客户端和服务器模型进行训练
    with tf.GradientTape() as tape_client, tf.GradientTape() as tape_server:
        # 客户端模型前向传播
        client_outputs = client_model_1(x, training=True)
        # 服务器模型前向传播
        server_logits = server_model_1(client_outputs, training=True)
        # 计算损失
        loss = loss_fn_1(y, server_logits)
    
    # 计算并应用服务器模型梯度
    grads_server = tape_server.gradient(loss, server_model_1.trainable_variables)
    optimizer_server_1.apply_gradients(zip(grads_server, server_model_1.trainable_variables))
    
    # 计算并应用客户端模型梯度
    grads_client = tape_client.gradient(loss, client_model_1.trainable_variables)
    optimizer_client_1.apply_gradients(zip(grads_client, client_model_1.trainable_variables))

    return loss

In [14]:

optimizer_client = tf.keras.optimizers.Adam()
optimizer_server = tf.keras.optimizers.Adam()
optimizer_client_1 = tf.keras.optimizers.Adam()
optimizer_server_1 = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
loss_fn_1 = tf.keras.losses.CategoricalCrossentropy(from_logits=True)


batch_size = 64
def augment(image, label):
    # 随机水平翻转图像
    image = tf.image.random_flip_left_right(image)
    # 随机调整亮度
    image = tf.image.random_brightness(image, max_delta=0.1)
    # 确保图像值仍然在0到1的范围内
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label

# 假设x_train, y_train, x_val, y_val, x_val_tigger, y_val_tigger已经定义并准备好了

# 训练数据集 - 应用数据增强
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.map(augment).shuffle(len(x_train)).batch(batch_size)

# 客户端验证数据集 - 也应用数据增强
client_1_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
client_1_dataset = client_1_dataset.map(augment).shuffle(len(x_val)).batch(batch_size)

# 服务器训练数据集 - 也应用数据增强


In [15]:
# 初始化TensorBoard回调
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
epochs = 60


2024-07-22 16:09:22.845792: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2024-07-22 16:09:22.845887: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1391] Profiler found 1 GPUs
2024-07-22 16:09:22.847563: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcupti.so.10.1
2024-07-22 16:09:22.852505: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1441] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES


In [16]:

for epoch in range(50):
    for x_batch, y_batch in train_dataset:
        loss = train_step_client(x_batch, y_batch)
    print(f"  Client Model Loss: {loss.numpy():.4f}")

2024-07-22 16:09:35.058835: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2024-07-22 16:09:36.113015: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2024-07-22 16:09:40.037663: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2024-07-22 16:09:40.524806: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


  Client Model Loss: 2.0039
  Client Model Loss: 1.8671
  Client Model Loss: 1.7893
  Client Model Loss: 1.7940
  Client Model Loss: 1.8263
  Client Model Loss: 1.7663
  Client Model Loss: 1.6844
  Client Model Loss: 1.7543
  Client Model Loss: 1.6635
  Client Model Loss: 1.6679
  Client Model Loss: 1.6076
  Client Model Loss: 1.6277
  Client Model Loss: 1.6897
  Client Model Loss: 1.6089
  Client Model Loss: 1.5833
  Client Model Loss: 1.5576
  Client Model Loss: 1.5932
  Client Model Loss: 1.5644
  Client Model Loss: 1.5671
  Client Model Loss: 1.5353
  Client Model Loss: 1.6131
  Client Model Loss: 1.5445
  Client Model Loss: 1.6068
  Client Model Loss: 1.5165
  Client Model Loss: 1.6159
  Client Model Loss: 1.5398
  Client Model Loss: 1.5186
  Client Model Loss: 1.6090
  Client Model Loss: 1.5598
  Client Model Loss: 1.4810
  Client Model Loss: 1.5442
  Client Model Loss: 1.4991
  Client Model Loss: 1.5327
  Client Model Loss: 1.5437
  Client Model Loss: 1.5553
  Client Model Loss:

In [17]:

for epoch in range(50):
    for x_batch, y_batch in client_1_dataset:   ##只修改这部分
        loss_1 = train_step_client_1(x_batch, y_batch)
    print(f"  Client Model 1 Loss: {loss_1.numpy():.4f}")

  Client Model 1 Loss: 2.0057
  Client Model 1 Loss: 2.0000
  Client Model 1 Loss: 2.0225
  Client Model 1 Loss: 1.7733
  Client Model 1 Loss: 2.0923
  Client Model 1 Loss: 1.7312
  Client Model 1 Loss: 1.8650
  Client Model 1 Loss: 1.7624
  Client Model 1 Loss: 1.8079
  Client Model 1 Loss: 1.8264
  Client Model 1 Loss: 1.8133
  Client Model 1 Loss: 1.9537
  Client Model 1 Loss: 1.9033
  Client Model 1 Loss: 1.9467
  Client Model 1 Loss: 1.8501
  Client Model 1 Loss: 1.8453
  Client Model 1 Loss: 1.8004
  Client Model 1 Loss: 1.5506
  Client Model 1 Loss: 1.7056
  Client Model 1 Loss: 1.7257
  Client Model 1 Loss: 1.6925
  Client Model 1 Loss: 1.7778
  Client Model 1 Loss: 1.7903
  Client Model 1 Loss: 1.7226
  Client Model 1 Loss: 1.6191
  Client Model 1 Loss: 1.6961
  Client Model 1 Loss: 1.5260
  Client Model 1 Loss: 1.6158
  Client Model 1 Loss: 1.6411
  Client Model 1 Loss: 1.7847
  Client Model 1 Loss: 1.5659
  Client Model 1 Loss: 1.5445
  Client Model 1 Loss: 1.5305
  Client M

In [18]:
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
def create_combined_model(client_model, server_model, compile_only=False):
    # Getting the input of the client model
    client_input = client_model.input
    
    # Getting the intermediate output by passing the input through the client model
    client_output = client_model(client_input)
    
    # The client model's output is used as the input for the server model
    server_output = server_model(client_output)
    
    # Defining a new model that chains the client and server models
    combined_model = Model(inputs=client_input, outputs=server_output)
    
    # Compile the combined model
    combined_model.compile(optimizer=Adam(),
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])
    
    if not compile_only:
        # If not compile_only, evaluate the model
        loss, accuracy = combined_model.evaluate(x_test, y_test, verbose=0)
        print(f"Test loss: {loss}, Test accuracy: {accuracy}")
    
    return combined_model


In [19]:


combined_model = create_combined_model(client_model, server_model, compile_only=True)
combined_model_1 = create_combined_model(client_model_1, server_model_1, compile_only=True)

In [20]:
loss, accuracy = combined_model.evaluate(x_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")


313/313 [==============================] - 5s 17ms/step - loss: 1.8588 - accuracy: 0.8226
Test loss: 1.8588007688522339, Test accuracy: 0.8226000070571899


In [21]:
loss, accuracy = combined_model_1.evaluate(x_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

313/313 [==============================] - 2s 6ms/step - loss: 3.2481 - accuracy: 0.6621
Test loss: 3.2481210231781006, Test accuracy: 0.6621000170707703


In [22]:
loss, accuracy = combined_model.evaluate(x_test_tigger, y_test_tigger)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")


313/313 [==============================] - 2s 6ms/step - loss: 35.3816 - accuracy: 0.0147
Test loss: 35.381629943847656, Test accuracy: 0.014700000174343586


In [23]:
class Autoencoder(tf.keras.Model):
    def __init__(self, input_shape):
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=input_shape),
            tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=2),
            tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2)
        ])
        self.decoder = tf.keras.Sequential([
            tf.keras.layers.Conv2DTranspose(16, kernel_size=3, strides=2, activation='relu', padding='same'),
            tf.keras.layers.Conv2DTranspose(32, kernel_size=3, strides=2, activation='relu', padding='same'),
            tf.keras.layers.Conv2D(128, (3, 3), activation='sigmoid', padding='same')
        ])

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

autoencoder = Autoencoder(input_shape=(32, 32, 128))  # 修改这里以符合实际输入维度


In [24]:
def train_step(x_batch, x_batch_1, client_model, client_model_1, autoencoder, optimizer):
    with tf.GradientTape() as tape:
        client_output = client_model(x_batch, training=True)
        client_output_1 = client_model_1(x_batch_1, training=True)
        reconstructed_output = autoencoder(client_output, training=True)
        loss = tf.reduce_mean(tf.keras.losses.mean_squared_error(client_output_1, reconstructed_output))
    gradients = tape.gradient(loss, autoencoder.trainable_variables + client_model.trainable_variables + client_model_1.trainable_variables)
    optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables + client_model.trainable_variables + client_model_1.trainable_variables))

    return loss


In [25]:
x_train_1 = np.tile(x_val_tigger, (4, 1, 1, 1)) 

In [26]:
train_dataset_2 = tf.data.Dataset.from_tensor_slices((x_test, x_test_tigger))
train_dataset_2 = train_dataset_2.shuffle(buffer_size=10000).batch(batch_size)  # 这里的10000是缓冲区大小
epochs = 20
batch_size = 32
optimizer = tf.keras.optimizers.Adam()

In [27]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    for (x_batch, x_batch_1) in train_dataset_2:
        # 在这里调用训练步骤
        # 确保检查x_batch和x_batch_1的维度是否符合模型的输入要求    
        # 假设 train_step 是您的训练函数
        loss = train_step(x_batch, x_batch_1, client_model, client_model_1, autoencoder, optimizer)
    print(loss)

Epoch 1/20
tf.Tensor(0.0022234828, shape=(), dtype=float32)
Epoch 2/20
tf.Tensor(0.0010940606, shape=(), dtype=float32)
Epoch 3/20
tf.Tensor(0.0013697436, shape=(), dtype=float32)
Epoch 4/20
tf.Tensor(0.00049138186, shape=(), dtype=float32)
Epoch 5/20
tf.Tensor(0.00036078936, shape=(), dtype=float32)
Epoch 6/20
tf.Tensor(0.00032208208, shape=(), dtype=float32)
Epoch 7/20
tf.Tensor(0.00020797105, shape=(), dtype=float32)
Epoch 8/20
tf.Tensor(0.0014761118, shape=(), dtype=float32)
Epoch 9/20
tf.Tensor(0.00088595, shape=(), dtype=float32)
Epoch 10/20
tf.Tensor(0.0002525195, shape=(), dtype=float32)
Epoch 11/20
tf.Tensor(0.00022820717, shape=(), dtype=float32)
Epoch 12/20
tf.Tensor(0.00088402204, shape=(), dtype=float32)
Epoch 13/20
tf.Tensor(0.00022062112, shape=(), dtype=float32)
Epoch 14/20
tf.Tensor(0.00018636361, shape=(), dtype=float32)
Epoch 15/20
tf.Tensor(0.00030117424, shape=(), dtype=float32)
Epoch 16/20
tf.Tensor(8.364693e-05, shape=(), dtype=float32)
Epoch 17/20
tf.Tensor(6.69

In [28]:
##后续正常模型的训练，直接微调
##自编码器只正常传播，不反向。


In [29]:
@tf.function
def train_step_2(data, labels):
    with tf.GradientTape(persistent=True) as tape:
        # 前向传播
        client_outputs = client_model(data, training=True)
        encoded_decoded = autoencoder(client_outputs, training=False)  # 训练时不更新autoencoder
        predictions = server_model(encoded_decoded, training=True)
        
        # 计算损失
        loss = tf.keras.losses.categorical_crossentropy(labels, predictions)

    # 计算梯度
    gradients_client = tape.gradient(loss, client_model.trainable_variables)
    gradients_server = tape.gradient(loss, server_model.trainable_variables)
    
    # 更新权重
    optimizer.apply_gradients(zip(gradients_client, client_model.trainable_variables))
    optimizer.apply_gradients(zip(gradients_server, server_model.trainable_variables))
    
    return loss

# 例如的优化器
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)


In [30]:
# 数据集和批次大小设定
batch_size = 32
epochs = 5
#(x_train, y_train), _ = tf.keras.datasets.cifar10.load_data()
#y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)

# 直接在循环中使用train_step
for epoch in range(epochs):
    for x_batch, y_batch in train_dataset:
        loss = train_step_2(x_batch, y_batch)
    print("Epoch: {}, Loss: {}".format(epoch, loss.numpy()))

Epoch: 0, Loss: [4.315202   0.8697642  2.512426   0.88232404 8.336674   0.3386597
 2.2156627  1.2124104  4.0026784  1.2269146  2.056648   4.2997613
 0.3414372  2.3001516  2.1645775  1.1601335  1.9990053  1.5417475
 1.8946757  1.753731   2.6477861  1.558814   4.9507484  0.46773475
 1.1523945  0.58854365 7.0830007  4.0500307  3.1773057  0.45554942
 0.5914203  0.3674245 ]
Epoch: 1, Loss: [2.4060707  0.9538797  2.4813142  1.1485956  4.0153146  0.6083218
 1.198568   2.0938106  2.557756   0.75932825 1.4817687  3.6660328
 0.55693966 1.2070912  1.1909689  0.8477852  1.3892384  1.8805768
 1.3879731  2.3299565  2.0131083  0.9278344  2.9840364  0.4279429
 1.2802446  0.5108975  4.068235   3.3829784  2.8055387  0.51726305
 0.74943745 0.68595666]
Epoch: 2, Loss: [1.8650084  1.0241531  2.4319594  1.0484667  2.858787   0.6318902
 0.6027827  2.1285548  2.246725   0.60246617 1.3054531  3.9627202
 0.44304463 1.0542877  0.83839756 0.7565739  1.4515008  1.973923
 1.3271011  2.7232187  1.6781708  0.85851294

In [31]:
loss, accuracy = combined_model.evaluate(x_test_tigger, y_test_tigger)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")


313/313 [==============================] - 2s 6ms/step - loss: 20.8177 - accuracy: 0.5631
Test loss: 20.817655563354492, Test accuracy: 0.5630999803543091


In [32]:
loss, accuracy = combined_model.evaluate(x_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

313/313 [==============================] - 2s 6ms/step - loss: 379.3007 - accuracy: 0.1112
Test loss: 379.3006896972656, Test accuracy: 0.1111999973654747


In [33]:
#313/313 [==============================] - 1s 3ms/step - loss: 0.9457 - accuracy: 0.7466
#Test loss: 0.9456651210784912, Test accuracy: 0.7465999722480774


## 